In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
img = '/kaggle/input'

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

After importing the images of the whales within the train/test set, for this step, the images will be extracted into pixel values for future computations. The feature of whale images will be classified using the basic l-layered logistic regression.

In [ ]:
import matplotlib.image as mpimg

Imaages either in RGB or Grayscale, hence to make every data consistent, one will attempt to convert every image to greyscale. Firstly, will use the library in importing the image plot from MATLAB to graph the image pixel values (greyscale). 

In [ ]:
def rgb_to_grey(img):
    if len(img.shape) == 2: #if there are only two scales then it is grayscale image
        return(img)
    greyImage = np.zeros(img.shape)
    Red = img[:,:,0] * 0.299
    Green = img[:,:,1] * 0.587
    Blue = img[:,:,2] * 0.114
    
    greyImage = Red + Green + Blue
    
    return(greyImage)
    

The next step is to standardize the image sizes, so all the images will have the consistent number of features. This is done by the following function. The constants that are multiplied for each images for each of the colors varied by the index number when converted to greyscaling. Reference: Stack Exchange

In [ ]:
def img_standardize(img, x_bin = 100, y_bin = 100):
    x_splits = np.linspace(0,img.shape[1] - 1, x_bins + 1, dtype = int)
    y_splits = np.linspace(0,img.shape[0] - 1, x_bins + 1, dtype = int)
    compressed = np.zeros((y_bins,x_bins))
    
    for i in range(y_bins):
        for j in range(x_bins):
            temp = np.mean(img[y_splits[i]:y_splits[i+1], x_splits[j]:x_splits[j+1]])
            if math.isnan(temp):
                if y_splits[i] == y_splits[i+1]:
                    compressed[i,j] = compressed[i-1,j]
                else:
                    compressed[i,j] = compressed[i,j-1]
            else: 
                compressed[i,j] = int(temp)
                
    return(compressed)


This process converts the .jpeg images into array of pixel values. Reference: Stack Exchange

In [ ]:
train_files_names = os.listdir(train_dir)[::5]
imgs_train = [rgb_to_grey(mpimg.imread(train_dir + '/' + file, format = 'JGP')) for file in os.listdir(train_dir)]




Remark: For the following step, I could not find a way to convert the image pixels into the train and test set yet, but the following will be the algorthm in making predictions and training the data set. For the **feature engineering** phase, I haven't have the knowledge to write the code, but for theoretical context, several procedures and techniques are appropriate for usage before the train/test set computation. 

The first pre-processing that was implemented above was the recoloring of the picture, in which the function takes the image that is supposedly RGB to a greyscale color scheme. This is for the ease of analysis, as every picture will be stardardize to commonscale. Afterwards, resizing is implemented to make all the images scaled into the same sizes, so every matrix of the layers when analyze in the logistic regression would be consistent (crucial as the future calculation will be based on vectorization). These image pixel numbers will be the train_x and test_x data sets. The third preprocessing phase would be the division of all the data sets by 255. Because the RGB-converted greyscale ranges from 1 to 255, every values are normalize to between 0 to 1 for ease of analysis (stats and probability are also in this same range).

In [ ]:
# Reshape the training and test examples 
train_x_flatten = imgs_train.reshape(train_x_orig.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten/255.
test_x = test_x_flatten/255.


This is the first phase of computation process of training set and test set. As mentioned, the identifying of the whale will be done thorugh l-layered logistic regression. In the first will initialize initial test/train set through random gaussian variable; this is because the initial data set regulated to be nonzero for the performance of the computation. The input variables are the n_x,n_h,n_y are the dimensions for the row, hidden, and columns of the training set, respectively. The beta would be the scale value that should be in the order of 10^-4 to 10^-2.

In [ ]:
def initialize_parameters(n_x, n_h, n_y, beta):
    W1 = np.random.randn(n_h,n_x) * beta
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y,n_h) * beta
    b2 = np.zeros((n_y, 1))
   
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters    

The above function would only randomize the variables for only for one single layer, hence this initialize_parameters_deep function would introduce the layer-dimensions into the data. The "W" and "b" of each layer-l is the weight and the bias for the training set. 

For **feature engineering**: use the basis of regularization and dropout. To prevent high variance the regularization of L-2 norm using the lambda/(the number of data sets x) * weight for each  of the linear_forward and linear_backward functions. But the "inverted dropout" can also be implemented with using the "keep_prob" parameter which will keep that amount of portion of the data for analyze. By eliminating the rest will decrese chance of overfitting: (with number of layer i):

d[i] = np.random.rand(a[i].shape[0], a[i].shape[1]) < keep_prob
a[i] = np.multiply(a3,d3)
a3 /= keep_prob #columns of d[i] that will be zero

In [ ]:
def initialize_parameters_deep(layer_dims):

    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
  
    return parameters

Linear_forward function performs the forward propagation. The output Z and cache will be used for the activation function in each of the iterations of calculation.

In [ ]:
def linear_forward(A, W, b):

    Z = np.dot(W,A) + b
   
    return Z, cache

The following function model will serve as an appendix for the computation sequences below. These functions are all positive definite function and will be used as an activation function for the forward process in the l-layered network.

In [ ]:
def sigmoid(z):
    s = 1/(1+np.exp(-z))
    return s

def tanh(z):
    s = (np.exp(2*x) - 1)/(np.exp(2*x) + 1)
    return s

def relu(z):
    s = np.maximum(0,z)
    return s

def softmax(z):
    expo = np.exp(z)
    expo_sum = np.sum(np.exp(z))
    return expo/expo_sum

linear_activation_forward takes in the iteration of the previous activation value A_prev, the weight, bias, and "activation" or certain activation function types. 

In [ ]:

def linear_activation_forward(A_prev, W, b, activation):
    #""""""
   # Arguments:
   # A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
   # W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
   # b -- bias vector, numpy array of shape (size of the current layer, 1)
   # activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

   # Returns:
   # A -- the output of the activation function, also called the post-activation value 
   # cache -- a python tuple containing "linear_cache" and "activation_cache";
   #          stored for computing the backward pass efficiently
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    elif activation == "hyptan":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = tanh(Z)
        
    elif activation == "softmax":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = softmax(Z)
    
    return A, cache

For the performance of the forward propagation, each of the functions in the appendix are utilized by each of the third of the data. The relu is best as the function increases the most at the beginning of the data. The hyperbolic tangent function will perform best when the number of data is not too large or too small. And the softmax is performed best near beginning and the end because of its own regression and the interpolation performances of the exponential functions.

In [ ]:
def L_model_forward(X, parameters):

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
        
    for l in range(1, L//3):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "softmax")
        caches.append(cache)
        
    for l in range(L//3, 2*L//3):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "hyptan")
        caches.append(cache)
        
    for l in range(2*L//3, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "softmax")
        caches.append(cache)

    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)
   
            
    return AL, caches

The compute_cost is the function that prepares the cost function to analyze the performance of the forward propagation. This is derived by the loss (Error) function from the sigmoid function. 

In [ ]:
def compute_cost(AL, Y):
  
    m = Y.shape[1]


    logprobs = np.multiply(np.log(AL),Y) +  np.multiply(np.log(1-AL), (1-Y))
    cost = -1/m*np.sum(logprobs)

    cost = np.squeeze(cost)     
    
    return cost

The linear_backward function calculates the gradient descent in which will iterate through the weight and bias parameters, introducing the dW and db, dZ. The three quantities are from the jacobian of the each W, b, Z function. 

In [ ]:
def linear_backward(dZ, cache):
 
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = 1./m*np.dot(dZ, A_prev.T)
    db = 1./m*np.sum(dZ, axis = 1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    

    return dA_prev, dW, db

Defines the activation function backwards, prepared using the four functions below: ReLU, Sigmoid, Hyperbolic Tangent, Softmax.

In [ ]:
def linear_activation_backward(dA, cache, activation):

    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "hyptan":
        dZ = hyptan_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "softmax":
        dZ = softmax_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

Use the L_model_backward to do a gradient descent for each hidden layers, utilizes the sigmoid and softmax function. 

In [ ]:
def L_model_backward(AL, Y, caches):

    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    
    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradients. 
    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
 
    for l in reversed(range(L-1)):
        # lth layer: (SOFTMAX -> LINEAR) gradients.
    
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)],  current_cache, activation = "softmax")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

This is the final function, which includes the other hyperparameters: learning rate, number of iterations, layer dimension to calculate the accuracy of the image. 

In [ ]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization. 
    parameters = initialize_parameters_deep(layers_dims)
   
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
    
        AL, caches = L_model_forward(X, parameters)
      
        # Compute cost.
    
        cost = compute_cost(AL, Y)
       
        # Backward propagation.
        grads = L_model_backward(AL, Y, caches)
   
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

Take the parameters and analyze the prediction for the function below.

In [ ]:
def predict(parameters, X):
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    A2, cache = forward_propagation(X,parameters)
    predictions = (A2 > 0.5)
 
    return predictions